In [2]:
# mnist_mlp_vs_logistic.py
# Keras MLP (dropout + early stopping) on MNIST, compared to a Logistic baseline.
# Saves confusion matrix for the MLP.

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping


def ensure_out():
    os.makedirs("outputs/mnist", exist_ok=True)


def main():
    ensure_out()

    # Load MNIST (uses ~/.keras/datasets/mnist.npz if offline)
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

    # ----- Logistic baseline -----
    X_train_flat = X_train.reshape((len(X_train), -1)).astype("float32") / 255.0
    X_test_flat = X_test.reshape((len(X_test), -1)).astype("float32") / 255.0

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_flat)
    X_test_scaled = scaler.transform(X_test_flat)

    logreg = LogisticRegression(
    solver="lbfgs", max_iter=100, n_jobs=-1
)

    logreg.fit(X_train_scaled, y_train)
    acc_lr = accuracy_score(y_test, logreg.predict(X_test_scaled))
    print(f"Logistic baseline test accuracy: {acc_lr:.4f}")

    # ----- MLP with dropout + early stopping -----
    X_train_n = X_train.astype("float32") / 255.0
    X_test_n = X_test.astype("float32") / 255.0

    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(512, activation="relu"),
        Dropout(0.3),
        Dense(256, activation="relu"),
        Dropout(0.3),
        Dense(10, activation="softmax"),
    ])
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    model.fit(X_train_n, y_train, validation_split=0.1,
              epochs=30, batch_size=128, callbacks=[es], verbose=0)

    test_loss, test_acc = model.evaluate(X_test_n, y_test, verbose=0)
    print(f"Keras MLP test accuracy: {test_acc:.4f}")

    # Confusion matrix for MLP
    y_pred_mlp = np.argmax(model.predict(X_test_n, verbose=0), axis=1)
    cm = confusion_matrix(y_test, y_pred_mlp)

    disp = ConfusionMatrixDisplay(cm, display_labels=np.arange(10))
    disp.plot(cmap="Blues", xticks_rotation=45)
    plt.title("MNIST Confusion Matrix - Keras MLP")
    plt.tight_layout()
    plt.savefig("outputs/mnist/cm_mlp.png", dpi=200)
    plt.close()
    print("Saved: outputs/mnist/cm_mlp.png")


if __name__ == "__main__":
    main()

Logistic baseline test accuracy: 0.9245


C:\diyyy\programs\python progs\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Keras MLP test accuracy: 0.9830
Saved: outputs/mnist/cm_mlp.png


In [3]:
# svm_kernels_margins_digits.py
# Digits dataset: Linear vs RBF SVM, grid search, PCA support-vector plots, accuracy heatmap.

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA


def ensure_out():
    os.makedirs("outputs/svm", exist_ok=True)


def heatmap(Z, Cs, gammas, title, fname):
    plt.figure()
    im = plt.imshow(Z, origin="lower", aspect="auto")
    plt.colorbar(im, label="CV Accuracy")
    plt.xticks(np.arange(len(gammas)), gammas)
    plt.yticks(np.arange(len(Cs)), Cs)
    plt.xlabel("gamma")
    plt.ylabel("C")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(fname, dpi=200)
    plt.close()


def plot_sv_2d(clf, X2d, y, title, fname):
    plt.figure()
    for c in np.unique(y):
        idx = y == c
        plt.scatter(X2d[idx, 0], X2d[idx, 1], s=10, edgecolors="k", alpha=0.6, label=str(c))
    sv = clf.support_vectors_
    plt.scatter(sv[:, 0], sv[:, 1], s=60, facecolors="none", edgecolors="r",
                linewidths=1.5, label="Support Vectors")

    plt.title(title)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend(loc="best", fontsize=7, ncol=2)
    plt.tight_layout()
    plt.savefig(fname, dpi=200)
    plt.close()


def main():
    ensure_out()

    X, y = load_digits(return_X_y=True)
    Xs = StandardScaler().fit_transform(X)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Baseline SVMs
    lin = SVC(kernel="linear", C=1.0)
    rbf = SVC(kernel="rbf", C=1.0, gamma="scale")

    lin_scores = cross_val_score(lin, Xs, y, cv=cv, scoring="accuracy")
    rbf_scores = cross_val_score(rbf, Xs, y, cv=cv, scoring="accuracy")

    print(f"Linear SVM CV accuracy: {lin_scores.mean():.4f} ± {lin_scores.std():.4f}")
    print(f"RBF SVM CV accuracy:    {rbf_scores.mean():.4f} ± {rbf_scores.std():.4f}")

    # Grid search (RBF)
    param_grid = {"C": [0.1, 1, 10, 100],
                  "gamma": [1e-3, 1e-2, 1e-1, 1.0]}

    grid = GridSearchCV(SVC(kernel="rbf"), param_grid=param_grid,
                        cv=cv, scoring="accuracy", n_jobs=-1)
    grid.fit(Xs, y)

    print(f"Best RBF params: {grid.best_params_}, best CV accuracy: {grid.best_score_:.4f}")

    Z = grid.cv_results_["mean_test_score"].reshape(len(param_grid["C"]),
                                                    len(param_grid["gamma"]))
    heatmap(Z, param_grid["C"], param_grid["gamma"],
            "RBF SVM Grid Search Accuracy",
            "outputs/svm/heatmap_rbf.png")
    print("Saved: outputs/svm/heatmap_rbf.png")

    # PCA + Support vector visualization
    Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.25,
                                          stratify=y, random_state=42)
    Xtr2 = PCA(n_components=2, random_state=42).fit_transform(Xtr)

    lin2 = SVC(kernel="linear", C=1.0).fit(Xtr2, ytr)
    rbf2 = SVC(kernel="rbf",
               C=grid.best_params_["C"],
               gamma=grid.best_params_["gamma"]).fit(Xtr2, ytr)

    plot_sv_2d(lin2, Xtr2, ytr, "Digits: Linear SVM Support Vectors (PCA-2D)", "outputs/svm/sv_linear.png")
    plot_sv_2d(rbf2, Xtr2, ytr, "Digits: RBF SVM Support Vectors (PCA-2D)", "outputs/svm/sv_rbf.png")

    print("Saved: outputs/svm/sv_linear.png, outputs/svm/sv_rbf.png")


if __name__ == "__main__":
    main()

Linear SVM CV accuracy: 0.9794 ± 0.0033
RBF SVM CV accuracy:    0.9839 ± 0.0060
Best RBF params: {'C': 10, 'gamma': 0.01}, best CV accuracy: 0.9827
Saved: outputs/svm/heatmap_rbf.png
Saved: outputs/svm/sv_linear.png, outputs/svm/sv_rbf.png


In [4]:
# iris_logreg_svm.py
# Logistic Regression & Linear SVM on Iris with standardization,
# 5-fold CV, confusion matrices, per-class metrics, and PCA(2D) plots.

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA


def ensure_out():
    os.makedirs("outputs/iris", exist_ok=True)


def pr_table(y_true, y_pred, labels):
    p, r, f1, sup = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, zero_division=0
    )
    print(f"{'class':>7} {'prec':>7} {'rec':>7} {'f1':>7} {'support':>8}")
    for c, pp, rr, ff, ss in zip(labels, p, r, f1, sup):
        print(f"{c:>7} {pp:7.3f} {rr:7.3f} {ff:7.3f} {ss:8d}")


def decision_plot_2d(model, X2d, y, title, fname, h=0.03):
    x_min, x_max = X2d[:, 0].min() - 1, X2d[:, 0].max() + 1
    y_min, y_max = X2d[:, 1].min() - 1, X2d[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.25)

    for c in np.unique(y):
        idx = (y == c)
        plt.scatter(X2d[idx, 0], X2d[idx, 1], s=20, edgecolors='k', label=str(c))

    plt.title(title)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend(fontsize=8)

    plt.tight_layout()
    plt.savefig(fname, dpi=200)
    plt.close()


def main():
    ensure_out()
    data = load_iris()
    X, y, names = data.data, data.target, data.target_names

    pipe_lr = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(multi_class="multinomial",
                                  solver="lbfgs", max_iter=1000))
    ])

    pipe_lsvm = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LinearSVC(C=1.0))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for label, model in [
        ("Logistic Regression", pipe_lr),
        ("Linear SVM", pipe_lsvm)
    ]:
        scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
        print(f"{label} 5-fold CV accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

        y_pred = cross_val_predict(model, X, y, cv=cv)
        cm = confusion_matrix(y, y_pred)

        print(f"\n{label} per-class metrics:")
        pr_table(y, y_pred, labels=[0, 1, 2])

        print("\nClassification report:")
        print(classification_report(y, y_pred, target_names=names, zero_division=0))

        disp = ConfusionMatrixDisplay(cm, display_labels=names)
        disp.plot(cmap="Blues", xticks_rotation=45)
        out = f"outputs/iris/cm_{label.replace(' ', '_').lower()}.png"
        plt.title(f"Iris Confusion Matrix - {label}")
        plt.tight_layout()
        plt.savefig(out, dpi=200)
        plt.close()
        print(f"Saved: {out}")

    # 2D PCA for visualization only
    X2d = PCA(n_components=2, random_state=42).fit_transform(X)

    lr2d = LogisticRegression(multi_class="multinomial",
                              solver="lbfgs", max_iter=1000).fit(X2d, y)
    decision_plot_2d(lr2d, X2d, y,
                     "Iris Decision Regions (PCA) - LogReg",
                     "outputs/iris/decision_lr.png")

    lsvm2d = LinearSVC(C=1.0).fit(X2d, y)
    decision_plot_2d(lsvm2d, X2d, y,
                     "Iris Decision Regions (PCA) - LinearSVM",
                     "outputs/iris/decision_lsvm.png")


if __name__ == "__main__":
    main()

C:\diyyy\programs\python progs\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\diyyy\programs\python progs\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\diyyy\programs\python progs\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\diyyy\programs\python progs\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' 

Logistic Regression 5-fold CV accuracy: 0.9533 ± 0.0452

Logistic Regression per-class metrics:
  class    prec     rec      f1  support
      0   1.000   1.000   1.000       50
      1   0.922   0.940   0.931       50
      2   0.939   0.920   0.929       50

Classification report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        50
  versicolor       0.92      0.94      0.93        50
   virginica       0.94      0.92      0.93        50

    accuracy                           0.95       150
   macro avg       0.95      0.95      0.95       150
weighted avg       0.95      0.95      0.95       150

Saved: outputs/iris/cm_logistic_regression.png
Linear SVM 5-fold CV accuracy: 0.9267 ± 0.0533

Linear SVM per-class metrics:
  class    prec     rec      f1  support
      0   1.000   0.980   0.990       50
      1   0.898   0.880   0.889       50
      2   0.885   0.920   0.902       50

Classification report:
              precisio

C:\diyyy\programs\python progs\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# sms_spam_tfidf_logreg.py
# TF-IDF + Logistic Regression spam detector. Prints precision/recall/F1 and
# top positive/negative features. Saves confusion matrix figure.
#
# If offline: put the UCI "SMSSpamCollection" file (tab-separated "label<TAB>message")
# next to this script and set LOCAL_PATH below. Otherwise the script tries a known URL.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, ConfusionMatrixDisplay
LOCAL_PATH = "SMSSpamCollection" # fallback local filename (optional)
REMOTE_TSV = "https://huggingface.co/datasets/DarkNeuron-AI/spam-sms-collection-01/resolve/main/spam.csv"
def load_data():
    if os.path.exists(LOCAL_PATH):
        return pd.read_csv(LOCAL_PATH, sep="\t", header=None, names=["label", "message"])
    try:
        df = pd.read_csv(REMOTE_TSV, encoding="latin1")  # default sep=','
        df = df.iloc[:, :2]  # keep only first two columns expected: label, message
        df.columns = ["label", "message"]
        return df
    except Exception as e:
        raise RuntimeError("Could not load SMS dataset. Place 'SMSSpamCollection' next to this script or ensure internet is available.") from e

def main():
    os.makedirs("outputs/sms", exist_ok=True)
    df = load_data()
    X_train, X_test, y_train, y_test = train_test_split(df["message"].values, df["label"].values,test_size=0.2, random_state=42, stratify=df["label"].values)
    print("Training labels:", np.unique(y_train))
    vect = TfidfVectorizer(lowercase=True, stop_words="english",ngram_range=(1,2),max_df=0.95, min_df=2)

    Xtr = vect.fit_transform(X_train)
    Xte = vect.transform(X_test)
    clf = LogisticRegression(penalty="l2", solver="liblinear",max_iter=1000)
    clf.fit(Xtr, y_train)
    y_pred = clf.predict(Xte)
    print("\nClassification report (test):")
    print(classification_report(y_test, y_pred, zero_division=0))
    print(f"Test accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=["ham","spam"])
    disp = ConfusionMatrixDisplay(cm, display_labels=["ham","spam"])
    disp.plot(cmap="Blues")
    plt.title("SMS Spam Confusion Matrix")
    plt.tight_layout()
    plt.savefig("outputs/sms/cm_sms.png", dpi=200); plt.close()
    print("Saved: outputs/sms/cm_sms.png")
    # Top positive/negative (toward "spam")
    feature_names = np.array(vect.get_feature_names_out())
    coefs = clf.coef_[0]
    top_pos = np.argsort(coefs)[-20:][::-1]
    top_neg = np.argsort(coefs)[:20]
    print("\nTop 20 SPAM-indicative features:")
    for f, w in zip(feature_names[top_pos], coefs[top_pos]):
        print(f"{f:30s} {w: .3f}")
    print("\nTop 20 HAM-indicative features:")
    for f, w in zip(feature_names[top_neg], coefs[top_neg]):
        print(f"{f:30s} {w: .3f}")

if __name__ == "__main__":
    main()